In [2]:
import sqlite3
import pandas as pd
con = sqlite3.connect(':memory:')

In [3]:
!git clone https://github.com/dsreload/sql_2

Cloning into 'sql_2'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 1), reused 4 (delta 1), pack-reused 1
Receiving objects: 100% (6/6), 1.92 MiB | 11.94 MiB/s, done.
Resolving deltas: 100% (1/1), done.


https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
data = pd.read_csv('sql_2/boston.csv')
data.to_sql('boston', con, index=False, if_exists='replace')

506

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [ ]:
q = 'select ' + ','.join(f"count(case when {c} is null then 1 end) as {c}" for c in data.columns) + " from boston"
pd.read_sql(q, con)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [ ]:
q = 'select ' + ','.join(f"count(distinct {c}) as {c}" for c in data.columns) + " from boston"
pd.read_sql(q, con)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,504,26,76,2,81,446,356,412,9,66,46,357,455,229


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [ ]:
for c in data.columns:
    q = f""" with stat as ( select {c}, row_number() over (order by {c}) rn, min({c}) over () min_{c}, count({c}) over () cnt from boston )
    select {c}, min_{c} from stat
    where rn = cnt/2
    """
    df = pd.read_sql(q, con)
    if df.at[0, c] == df.at[0, 'min_' + c]:
        print(f"{c} {df.at[0, c]}") # у данных полей более половины значений равны минимуму

ZN 0.0
CHAS 0.0


4. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [ ]:
q = f"""select lstat, medv, dense_rank() over (order by lstat DESC) rk from boston order by 2 desc"""
pd.read_sql(q, con) # чем дороже дом, тем выше статус населения

,LSTAT,MEDV,rk
0,9.53,50.0,280
1,8.88,50.0,301
2,7.44,50.0,334
3,5.12,50.0,397
4,4.63,50.0,414
...,...,...,...
501,23.97,7.0,41
502,29.97,6.3,12
503,26.77,5.6,26
504,30.59,5.0,11


5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [ ]:
pd.read_sql('''select chas, avg("medv") from boston group by 1''', con,)

,CHAS,"avg(""medv"")"
0,0.0,22.093843
1,1.0,28.440000


6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [ ]:
pd.read_sql("""select avg(crim), avg(zn), avg(indus), avg(nox) from boston group by chas""", con,
).T.rename(columns={0: 'away', 1:'near'}).query('near > away') # домов и оксида азота у реки в среднем больше, возможно, рядом с рекой производство

,away,near
avg(indus),11.019193,12.719143
avg(nox),0.551817,0.593426


7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [16]:
pd.read_sql('''select nox, indus from
(select  max(nox) nox, npct from (select nox, (ntile(10) over (order by nox)) 'npct' from boston)  group by 2) n join
(select  max(indus) indus, ipct from (select indus, (ntile(10) over (order by indus)) 'ipct' from boston)  group by 2) i
on n.npct = i.ipct
''', con, ) # корреляция есть

,nox,indus
0,0.426,2.89
1,0.442,4.39
2,0.472,5.96
3,0.507,7.38
4,0.538,9.69
5,0.580,13.89
6,0.609,18.10
7,0.671,18.10
8,0.713,19.58
9,0.871,27.74
